In [245]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from scipy import stats
import numpy as np

In [246]:
df_us = pd.read_csv('./to_annotate/nli-labels-authors.csv')
#df_us = pd.read_csv('./to_annotate/nli-bach.csv')
#df_us = pd.read_csv('./to_annotate/nli-paul.csv')
AGG_SCORES = True

df_us.head()

,original_sentence1,contrast text,original_sentence2,original label,contrast label,part,flip_label_human,minimal_change_human,distribution_human,Unnamed: 9
0,A group of swimmers jump into a pool.,A group of swimmers are jumping into a pool.,A group of swimmers are sitting on an airplane...,contradiction,entailment,revised_hypothesis,4,3,4,NaN
1,A white greyhound dog wearing a muzzle runs ar...,A white dog wearing a muzzle without a muzzle ...,The dog is not wearing a muzzle.,contradiction,entailment,revised_hypothesis,2,-1,-1,Example for the paper
2,a woman wearing a pink tank top holding a mug ...,a woman wearing a blue tank top holding a car,A woman in a blue tank top holding a car.,contradiction,entailment,revised_hypothesis,1,-1,-1,NaN
3,"a boy wearing jean shorts, and a black and blu...",The boy reads a book.,The boy plays a video game.,contradiction,entailment,revised_hypothesis,4,4,4,NaN
4,A man in a green shirt and large beige hat is ...,A worker in a uniform is at the office.,the man is at home sleeping,contradiction,entailment,revised_hypothesis,2,-1,-1,NaN


In [247]:
df_gpt = pd.read_csv('./to_annotate/nli.csv')
df_gpt.head()

,Unnamed: 0,original_sentence1,original_sentence2,original label,contrast label,contrast text,flip_label,minimal_change_1,minimal_change_2,distribution,LLM,part,Unnamed: 0.1,closest_instance
0,673,A young boy with a shaved head and dirty white...,The young boy went swimming.,contradiction,entailment,A young boy with a shaved head and dirty white...,1.0,2.0,4.0,2.0,crowd,revised_premise,NaN,NaN
1,769,A person in a red shirt is mowing the grass wi...,A person in red is swimming in the Olympics.,contradiction,entailment,A person in a red shirt is swimming in the Oly...,4.0,4.0,4.0,4.0,crowd,revised_premise,NaN,NaN
2,129,A boy dressed for summer in a green shirt and ...,A boy alien dressed for summer in a green shir...,entailment,neutral,A boy alien dressed for summer in a green t-sh...,1.0,2.0,3.0,3.0,crowd,revised_hypothesis,NaN,NaN
3,798,A photographer and assistant take a photo of a...,The photographer is taking a photo of a dog.,contradiction,neutral,The photographer is taking a large photo of a ...,4.0,4.0,4.0,4.0,crowd,revised_hypothesis,NaN,NaN
4,431,A red truck is parked next to a burning blue b...,The burning blue building will turn to dust af...,neutral,entailment,A red truck is parked next to a burning blue b...,1.0,2.0,3.0,2.0,llama2_70b,revised_hypothesis,431.0,Flaming are shooting from the rear of a race c...


In [248]:
merged = df_us.merge(df_gpt, on=['original_sentence1', 'original_sentence2', 'contrast text', 'original label', 'contrast label'])

In [249]:
def aggreate_scores(df, c):
    def change_score(s):
        if s == 2:
            return 1
        elif s == 3:
            return 4
        else:
            return s
    df[c] = df.apply(lambda x: change_score(x[c]), axis=1)
    return df

In [250]:
if AGG_SCORES:
    merged = aggreate_scores(merged, 'flip_label_human')
    merged = aggreate_scores(merged, 'flip_label')

In [251]:
merged['flip_label'].unique()

array([4., 1.])

In [252]:
merged['flip_label_human'].unique()

array([4, 1])

In [253]:
for p in merged['part_x'].unique():
    print(p)
    tmp = merged[merged['part_x'] == p]
    us_fl = tmp['flip_label_human'].values
    gpt_fl = tmp['flip_label'].values


    assert len(us_fl) == len(gpt_fl)

    print(len(us_fl))

    print('cohen kappa :', np.round(cohen_kappa_score(us_fl, gpt_fl), 2))
    res = stats.spearmanr(us_fl, gpt_fl)
    print('spearman corr.: ', np.round(res.statistic, 2))

revised_hypothesis
14
cohen kappa : 0.57
spearman corr.:  0.58
revised_premise
14
cohen kappa : 0.0
spearman corr.:  0.0


In [254]:
us_fl = merged['flip_label_human'].values
gpt_fl = merged['flip_label'].values


assert len(us_fl) == len(gpt_fl)

print(len(us_fl))

print('cohen kappa :', np.round(cohen_kappa_score(us_fl, gpt_fl), 2))
res = stats.spearmanr(us_fl, gpt_fl)
print('spearman corr.: ', np.round(res.statistic, 2))

28
cohen kappa : 0.29
spearman corr.:  0.29
